In [1]:
# %%
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import math
from tqdm import tqdm
from multiprocessing import Pool
import multiprocessing as mp
import copy
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from datetime import datetime
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os
import torchmetrics
import pickle
import time


global ff_dim
size = 20
type = "NR"
ff_dim = 1024


In [2]:
a = torch.rand(10)
b = a + 5

In [4]:
def f1(a, b):
    diff = []
    for i,j in zip(a,b):
        gap = j.item() - i.item()
        diff.append(gap/i.item())
    
    return np.mean(diff)*100

f1(a,b)


1197.6054044463294

In [5]:
def f2(a,b):
    gaps = b-a

    return torch.mean(torch.div(gaps,a)).item()*100

f2(a,b)

1197.6054191589355

In [ ]:
# %%
class Net(nn.Module):
    
    def __init__(self,TSP_size,ff_dim):
      
        self.TSP_size = TSP_size
        self.ff_dim = ff_dim
        self.n_layers = math.ceil(math.log(TSP_size,2))

        super(Net, self).__init__()

        self.encoder = nn.TransformerEncoderLayer(d_model=self.TSP_size, nhead=self.TSP_size//10, batch_first=True, dim_feedforward=self.ff_dim)
        self.transformer = nn.TransformerEncoder(self.encoder,num_layers=self.n_layers)      



    def forward(self,x):
      
        x = F.relu(x)
        
        out1 = self.transformer(x)
        out2 = torch.transpose(out1, 1, 2)
        x1 = F.softmax(out1,1)
        x2 = F.softmax(out2,2)
        x3 = torch.add(x1, x2)
        x4 = F.hardtanh(x3, 0, 1)

        return x4



def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def is_symmetric(matrix):
    # Check if the matrix is equal to its transpose
    return torch.allclose(matrix, matrix.t())
    

# %%
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.tsp_size = (data.shape[1]-1)//3
        self.data = data[:,:2*self.tsp_size]
        self.lengths = data[:,-1]
        self.labels = data[:,2*self.tsp_size:3*self.tsp_size]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample_data = self.data[idx]
        sample_label = self.labels[idx]
        sample_lengths = self.lengths[idx]

        if self.transform:
            sample_data,sample_label,sample_raw_dist = self.transform((sample_data,sample_label))
            

        return sample_data, sample_label, sample_raw_dist, sample_lengths

class ToTensor(object):
    def __call__(self, data):

        sample_data = data[0]
        sample_label = data[1]
        
        size = sample_data.shape[0]//2
        X = np.column_stack((sample_data[:size], sample_data[size:]))
        dist = euclidean_distances(X,X)
        dist_ = dist/np.max(dist)
        dist_ = 1-dist_
        np.fill_diagonal(dist_, 0)
        dist_ = torch.tensor(dist_, dtype=torch.float32)
        dist = torch.tensor(dist, dtype=torch.float32)

        mroutelist = sample_label.tolist()
        mroutelist.append(0)
        route_matrix = np.zeros((size,size))
        
        for i in range(size):
            origin = int(mroutelist[i])
            dest = int(mroutelist[i+1])
            route_matrix[origin,dest] = 1
            route_matrix[dest,origin] = 1
     

        route_matrix = torch.tensor(route_matrix, dtype=torch.float32)

        return dist_, route_matrix, dist



# %%
data = np.loadtxt("BenchmarkInstances/{}_{}.csv".format(type, size), delimiter=',')

#train, val = train_test_split(data, random_state=42, test_size=0.05)

# %%
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# %%

#train_dataset = CustomDataset(train, transform=ToTensor())
#train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_cores)

val_dataset = CustomDataset(data, transform=ToTensor())
val_data_loader = DataLoader(val_dataset, batch_size = len(data), shuffle=False, num_workers=num_cores)

# %%
model = Net(size,ff_dim=ff_dim)
print(count_parameters(model))
model.to(device)


In [ ]:
model.load_state_dict(torch.load("best_model_{}.pth".format(size), map_location=torch.device('cpu')))
model.eval()


In [ ]:
def greedy_decode_batch(output_matrix, distance_matrix):
    batch_size, num_points, _ = output_matrix.size()

    # Initialize the tour tensor
    best_tour = torch.zeros(batch_size, num_points, dtype=torch.long)

    for i in range(batch_size):
       output_matrix[i].fill_diagonal_(0)


    best_tour_length = torch.full((batch_size,), float('inf'))

    for start_point in range(num_points):
        # Start from the current point for each instance in the batch
        current_points = torch.full((batch_size,), start_point, dtype=torch.long)
        visited_points = torch.zeros(batch_size, num_points, dtype=torch.bool)
        tour = torch.zeros(batch_size, num_points, dtype=torch.long)
        tour[torch.arange(batch_size),0] = current_points

        # Perform greedy decoding for the current starting point
        for step in range(num_points-1):
            visited_points[torch.arange(batch_size), current_points] = True
            probabilities = output_matrix[torch.arange(batch_size), current_points, :]
            probabilities[visited_points] = 0
            # Choose the next point with the highest probability
            next_points = torch.argmax(probabilities, dim=-1)
            tour[:, step+1] = next_points
            
            current_points = next_points

        # Calculate the tour length for each sample in the batch
        tour_lengths = torch.sum(distance_matrix[torch.arange(batch_size).unsqueeze(1), tour, torch.roll(tour, shifts=-1, dims=1)], dim=-1)
        # Update the best tour if the current one has a shorter length
        update_mask = tour_lengths < best_tour_length
        best_tour[update_mask] = tour[update_mask]
        best_tour_length[update_mask] = tour_lengths[update_mask]

    return best_tour, best_tour_length


def greedy_total(output_matrix, normalized_dist, distance_matrix):

    pred_tour, pred_tour_length = greedy_decode_batch(output_matrix, distance_matrix)

    sum_tour, sum_tour_length = greedy_decode_batch(1-normalized_dist + output_matrix, distance_matrix)

    mult_tour, mult_tour_length = greedy_decode_batch((1 - normalized_dist) * output_matrix, distance_matrix)


    all_tour_lengths = torch.stack([pred_tour_length, sum_tour_length, mult_tour_length], dim=-1)
    all_tours = torch.stack([pred_tour, sum_tour, mult_tour], dim=-1)

    _, min_indices = torch.min(all_tour_lengths, dim=-1)
    min_tour_lengths = torch.gather(all_tour_lengths, dim=-1, index=min_indices.unsqueeze(-1)).squeeze(-1)
    min_tours = all_tours[torch.arange(all_tours.size(0)).unsqueeze(1), :, min_indices.unsqueeze(-1)]

    return min_tours.squeeze(1), min_tour_lengths


In [ ]:

def calculate_tour_length(distance_matrix, tour):
    
    batch_size, num_points, _ = distance_matrix.size()
    
    # Create a tensor to hold the tour lengths for each batch

    tour_lengths = torch.sum(distance_matrix[torch.arange(batch_size).unsqueeze(1), tour, torch.roll(tour, shifts=-1, dims=1)], dim=-1)


    return tour_lengths

def two_opt_swap(tour, i, j):
    # Perform 2-opt swap between edges (i, i+1) and (j, j+1)
    new_tour = tour.clone()
    new_tour[:, i:j+1] =torch.flip(tour[:, i:j+1], dims=[1])
    return new_tour

def two_opt(distance_matrix, tour):
    batch_size, num_points, _ = distance_matrix.size()

    # Calculate the initial tour length
    initial_tour_length = calculate_tour_length(distance_matrix, tour)

    # Perform 2-opt swaps
    for i in range(1, num_points - 2):
        for j in range(i + 1, num_points):
            # Calculate the new tour length after the 2-opt swap
            new_tour = two_opt_swap(tour, i, j)
            new_tour_length = calculate_tour_length(distance_matrix, new_tour)
            # Identify tours with shorter lengths
            improve_mask = new_tour_length < initial_tour_length
            
            # Update tours and lengths for improved cases
            tour[improve_mask] = new_tour[improve_mask]
            initial_tour_length[improve_mask] = new_tour_length[improve_mask]

    return tour, initial_tour_length


In [ ]:
def GreedyDecoder(val_data_loader, model):

    my_dct = {}

    start = time.time()

    for i,batch in enumerate(val_data_loader):

        with torch.no_grad():

            preds = model(batch[0].to(device))

            lns = batch[3]

            decoded_tour, decoded_length = greedy_total(preds,batch[0],batch[2])

            two_opt_tour, two_opt_length = two_opt(batch[2], decoded_tour)

            value_gap = two_opt_length.detach() - lns.detach()

            init_percentage_gap = torch.mean((value_gap/lns))*100

            print("init gap: ", init_percentage_gap)

            tol = 1000000

            while tol > 0.00001:

                two_opt_tour, two_opt_length = two_opt(batch[2], decoded_tour)

                value_gap = two_opt_length-lns
                new_percentage_gap = torch.mean((value_gap/lns))*100
                print("new gap: ", new_percentage_gap)
                tol = init_percentage_gap - new_percentage_gap

                print(tol)

                init_percentage_gap = new_percentage_gap

    end = time.time()

    my_dct["mean_gap"] = new_percentage_gap
    my_dct["min_gap"] = 100*(torch.min(torch.div(two_opt_length, lns))-1).item()
    my_dct["max_gap"] = 100*(torch.max(torch.div(two_opt_length, lns))-1).item()
    my_dct["values"] = 100*(torch.div(two_opt_length, lns)-1).numpy()
    my_dct["solution_time"] = end-start 
    
    return my_dct



In [ ]:
def RunBenchmarks(instance_type, instance_size):

    model = Net(instance_size,ff_dim=ff_dim)

    model.to(device)    
    
    data = np.loadtxt("BenchmarkInstances/{}_{}.csv".format(instance_type, instance_size), delimiter=',')

    val_dataset = CustomDataset(data, transform=ToTensor())
    
    val_data_loader = DataLoader(val_dataset, batch_size = len(val_dataset), shuffle=False, num_workers=1)

    return GreedyDecoder(val_data_loader, model)
    
    



In [ ]:
RunBenchmarks("NS", 20)

In [ ]:
instance_names = ["G1", "G2_2", "G3_1", "G3_2", "G3_3", "G4", "SG", "US", "UR", "NS", "NR"]
sizes = [20,50,100]
parameter_list = []
for item1 in instance_names:
    for item2 in sizes:
        parameter_list.append({"instance_type":item1, "instance_size":item2})


In [ ]:
def RunBenchmarks(instance_type, instance_size):

    model = Net(instance_size,ff_dim=ff_dim)

    model.to(device)    
    
    data = np.loadtxt("BenchmarkInstances/{}_{}.csv".format(instance_type, instance_size), delimiter=',')

    val_dataset = CustomDataset(data, transform=ToTensor())
    
    val_data_loader = DataLoader(val_dataset, batch_size = len(), shuffle=False, num_workers=mp.cpu_count())

    return GreedyDecoder(val_data_loader, model)
    
    



In [ ]:
for i,batch in enumerate(val_data_loader):
    with torch.no_grad():
        preds = model(batch[0].to(device))
        labels = batch[1]
        dist_mat = batch[2]
        lns = batch[3]
        decoded_tour, decoded_length = greedy_total(preds,batch[0],dist_mat)
        two_opt_tour, two_opt_length = two_opt(dist_mat, decoded_tour)
        two_opt_tour, two_opt_length = two_opt(dist_mat, two_opt_tour)
        two_opt_tour, two_opt_length = two_opt(dist_mat, two_opt_tour)
        #two_opt_tour, two_opt_length = two_opt(dist_mat, decoded_tour)
        #two_opt_tour, two_opt_length = two_opt(dist_mat, decoded_tour)
        #two_opt_tour, two_opt_length = two_opt(dist_mat, decoded_tour)


In [ ]:
instance_names = ["G1", "G2_2", "G3_1", "G3_2", "G3_3", "G4", "SG", "US", "UR", "NS", "NR"]
res_list = []
for item in instance_names:
    res_list.append(GreedyDecoder_(20, item))

In [7]:
data = pickle.load(open("eval.pkl", "rb"))

In [9]:
data[100,'G1']

{'time': 32.959229946136475,
 'mean_gap': 5.629386329111288,
 'max_gap': 16.724353458385053,
 'min_gap': -1.2219485423514698e-07,
 'perf': array([0.06413242, 0.05384084, 0.11348824, ..., 0.10274092, 0.01506078,
        0.06232909])}